In [106]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
from dash.dash_table import DataTable
from datetime import date
import psycopg2

In [107]:
def get_db_connection():
    return psycopg2.connect(
        host="localhost",
        dbname = "school_fee_db",
        user = "uthkarsh",
        password = "Ruthwik081@")

In [108]:
APP_STYLE = {
    'font-family': 'Arial, sans-serif',
    'margin': '0 auto',
    'padding': '20px',
    'max-width': '800px',
    'background-color': '#ffffff',  # Softer white background
    'box-shadow': '0 4px 8px rgba(0, 0, 0, 0.1)',
    'border-radius': '10px',
    'padding-bottom': '30px',
}

HEADER_STYLE = {
    'text-align': 'center',
    'padding': '15px 0',
    'background-color': '#0066cc',  # Calming blue header
    'color': 'white',
    'border-radius': '10px 10px 0 0',
    'font-size': '24px',
    'font-weight': 'bold'
}

SUB_HEADER_STYLE = {
    'text-align': 'center',
    'font-size': '24px',
    'font-weight': 'bold'    
}

NAV_BAR_STYLE = {
    'text-align': 'center',
    'margin': '15px 0',
}

NAV_LINK_STYLE = {
    'margin': '0 10px',
    'padding': '10px 20px',
    'background-color': '#0066cc',  # Matches header
    'color': 'white',
    'border-radius': '5px',
    'text-decoration': 'none',
    'font-weight': 'bold',
    'box-shadow': '0 2px 4px rgba(0, 0, 0, 0.2)',  # Adds depth
}

SECTION_STYLE = {
    'padding': '15px 20px',
    'display': 'flex',
    'flex-direction': 'column',
    'align-items': 'center',
}

BUTTON_STYLE = {
    'background-color': '#0066cc',  # Matches header and nav
    'color': 'white',
    'border': 'none',
    'padding': '10px 30px',
    'border-radius': '5px',
    'cursor': 'pointer',
    'font-size': '16px',
    'margin-top': '15px',
    'box-shadow': '0 2px 4px rgba(0, 0, 0, 0.2)',  # Subtle shadow for depth
}

INPUT_STYLE = {
    'margin': '10px 0',
    'padding': '12px 15px',
    'width': '100%',
    'border': '1px solid #ccc',
    'border-radius': '5px',
    'font-size': '14px',
    'box-shadow': 'inset 0 1px 3px rgba(0, 0, 0, 0.1)',  # Adds dimension
}

DROPDOWN_STYLE = {
    'margin': '10px 0',
    'padding': '12px 15px',
    'width': '100%',
    'height': '50%',
    'border': '1px solid #ccc',
    'border-radius': '1px',
    'font-size': '14px',
    'background-color': '#f9f9f9',  # Softer background to match input
    'box-shadow': 'inset 0 1px 3px rgba(0, 0, 0, 0.1)',  # Same as input
}

DATE_STYLE = {
    'display': 'flex',
    'justify-content': 'space-between',
    'gap': '15px',  # Balanced spacing between date fields
    'width': '100%',
}

DATE_INPUT_STYLE = {
    'padding': '12px 15px',
    'flex': '1',  # Ensures even width for both date inputs
    'border': '1px solid #ccc',
    'border-radius': '5px',
    'font-size': '14px',
    'box-shadow': 'inset 0 1px 3px rgba(0, 0, 0, 0.1)',
}


In [109]:
def student_details_layout():
    input_fields = [
    {'id': 'admission_number', 'type': 'number', 'placeholder': 'Admission Number', 'value': None},
    {'id': 'aadhar_number', 'type': 'number', 'placeholder': 'Aadhar Number', 'value': None},
    {'id': 'StudentName', 'type': 'text', 'placeholder': 'Student Name', 'value': ''},
    {'id': 'FatherName', 'type': 'text', 'placeholder': "Father's Name", 'value': ''},
    {'id': 'contact_number', 'type': 'tel', 'placeholder': 'Contact Number', 'value': ''},
    {'id': 'village', 'type': 'text', 'placeholder': 'Village', 'value': ''}
    ]

    date_pickers = [
    {'id': 'dob', 'placeholder': 'Date of Birth'},
    {'id': 'doj', 'placeholder': 'Date of Joining'}
    ]

    return html.Div([
        html.H1("Student Details Entry", style=HEADER_STYLE),
        html.Div([
            *[dcc.Input(id=field['id'], type=field['type'], placeholder=field['placeholder'],value=field['value'], style=INPUT_STYLE) for field in input_fields], # Use list comprehension for inputs
            dcc.Dropdown(
                id='gender',
                options=[{'label': label, 'value': label} for label in ['Boy', 'Girl']], # Simplified options
                placeholder="Gender",
                style=DROPDOWN_STYLE
            ),
            html.Div([
                *[dcc.DatePickerSingle(id=date['id'], display_format='DD/MM/YYYY', placeholder=date['placeholder'], style=DATE_INPUT_STYLE) for date in date_pickers] # List comprehension for date pickers
            ], style=DATE_STYLE),
            html.Button('Submit', id='student-details-submit-button', n_clicks=0, style=BUTTON_STYLE),
        ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [110]:
# Page 2: Student Class Details Layout
def student_class_details_layout():
    return html.Div([
        html.H1("Student Class Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='admission_number', type='number', placeholder='Admission Number', style=INPUT_STYLE),
            dcc.Input(id= 'roll_number', type= 'number', placeholder= 'Roll Number', style=INPUT_STYLE),
            dcc.Input(id= 'photo_id', type= 'number', placeholder= 'Photo ID',value= None, style=INPUT_STYLE),
            dcc.Dropdown(
                id='class_no', 
                options=[{'label': f'Class {i}', 'value': i} for i in range(1, 13)],
                placeholder="Class",
                style=DROPDOWN_STYLE
            ),
            dcc.Input(id='section', type='text', placeholder='Section', style=INPUT_STYLE),
            dcc.Input(id='current_year', type='number', placeholder='Year', style=INPUT_STYLE),
            dcc.Dropdown(id='enrolled', options=[
                {'label': 'Yes', 'value': 'Yes'},
                {'label': 'No', 'value': 'No'}
            ], placeholder="Currently Enrolled", style=DROPDOWN_STYLE),
            dcc.Dropdown(id='language', options=[
                {'label': 'Telugu', 'value': 'Telugu'},
                {'label': 'Hindi', 'value': 'Hindi'},
                {'label': 'Sanskrit', 'value': 'Sanskrit'}
            ], placeholder="Language", style=DROPDOWN_STYLE),
            dcc.Dropdown(id='vocational', options=[
                {'label': 'Agriculture', 'value': 'Agriculture'},
                {'label': 'Artificial Intelligence', 'value': 'AI'},
                {'label': 'Physical Activity Trainer', 'value': 'PAT'},
                {'label': 'Tourism', 'value': 'Tourism'}
            ], placeholder="Vocational", style=DROPDOWN_STYLE),
            html.Button('Submit', id='student-class-submit-button', n_clicks=0, style=BUTTON_STYLE),
        ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [111]:
# Page 3: Fee Details Layout
def fee_details_layout():
    return html.Div([
        html.H1("Fee Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='admission_number', type='number', placeholder='Admission Number', style=INPUT_STYLE),
            dcc.Input(id='year', type='number', placeholder='Year', style=INPUT_STYLE),
            dcc.Input(id='school_fee', type='number', placeholder='School Fee', style=INPUT_STYLE),
            dcc.Dropdown(
                id='school_fee_concession_reason',
                options=[
                    {'label': reason, 'value': reason} for reason in [
                        'Staff', 'Sibling', 'OTP', 'TF', 'FP', 'EC', 'SC', 'General'
                    ]
                ],
                placeholder="School Fee Concession Reason",
                style=DROPDOWN_STYLE
            ),
            dcc.Dropdown(
                id='transport_used',
                options=[
                    {'label': 'Yes', 'value': 'Yes'},
                    {'label': 'No', 'value': 'No'}
                ],
                placeholder="Transport Used",
                style=DROPDOWN_STYLE
            ),
            dcc.Input(id='transport_fee', type='number', placeholder='Transport Fee', style=INPUT_STYLE, disabled=True),
            dcc.Input(id='transport_fee_concession', type='text', placeholder='Transport Fee Concession', style=INPUT_STYLE, disabled=True),
            dcc.Input(id='application_fee', type='number', placeholder='Application Fee', style=INPUT_STYLE),
            html.Button('Submit', id='fee-details-submit-button', n_clicks=0, style=BUTTON_STYLE),
            ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [112]:
# Page 4: Fee Details Layout
def transport_details_layout():
    return html.Div([
        html.H1("Transport Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='pickup_point', type='text', placeholder='Pick-Up Point', style=INPUT_STYLE),
            dcc.Input(id='transport_route', type='number', placeholder='Route Number', style=INPUT_STYLE),
           html.Button('Submit', id='transport-details-submit-button', n_clicks=0, style=BUTTON_STYLE),
            ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [113]:
# Page 5: Fee Payment Details Layout
def fee_type_selection_layout():
    input_fields = [
        {'id': 'paid', 'type': 'number', 'placeholder': 'Admission Number'},
        {'id': 'due', 'type': 'number', 'placeholder': 'Aadhar Number'},
        {'id': 'receipt_number', 'type': 'number', 'placeholder': 'Student Name'}
    ]
    return html.Div([
        html.H1("Fee Payment Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='admission_number', type='number', placeholder='Admission Number', style=INPUT_STYLE),
            dcc.Input(id='year', type='number', placeholder='Year', style=INPUT_STYLE),
            dcc.Dropdown(
                id='fee_type',
                options=[
                    {'label': 'School Fee', 'value': 'school_fee'},
                    {'label': 'Transport Fee', 'value': 'transport_fee'},
                    {'label': 'Application Fee', 'value': 'application_fee'}
                ],
                value=None,  # Default to None
                placeholder="Select Fee Type",
                style=DROPDOWN_STYLE
            ),
            dcc.Dropdown(
                id='fee_term',
                options=[
                    {'label': 'Term 1', 'value': 'term_1'},
                    {'label': 'Term 2', 'value': 'term_2'},
                    {'label': 'Term 3', 'value': 'term_3'}
                ],
                value=None,  # Default to None
                placeholder="Select Fee Term",
                style=DROPDOWN_STYLE
            ),
            *[dcc.Input(id = field['id'], type = field['type'], placeholder = field['placeholder'], style=INPUT_STYLE) for field in input_fields],
            html.H2("Date paid", style= SUB_HEADER_STYLE),
            dcc.DatePickerSingle(
                id='paid_date',
                display_format='DD/MM/YYYY',
                placeholder="Date Paid",
                date=date.today(),  # Set the default date to today
                style={'margin-top': '10px'}
            ),
            html.Button('Next', id='next-button', n_clicks=0, style=BUTTON_STYLE),
            html.Div(id='fee-content'),
        ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [114]:
# Page 6: View Records Layout
def view_records_layout():
    return html.Div([
        html.H1("View Records", style=HEADER_STYLE),
        html.Div([
            dcc.Dropdown(
                id='view-class', 
                options=[{'label': f'Class {i}', 'value': i} for i in range(1, 13)],
                placeholder="Select Class",
                style=DROPDOWN_STYLE
            ),
            dcc.Dropdown(
                id='view-year', 
                options=[{'label': str(year), 'value': year} for year in range(2020, 2025)],
                placeholder="Select Year",
                style=DROPDOWN_STYLE
            ),
            DataTable(
                id='student-table',
                columns=[
                    {'name': 'Admission Number', 'id': 'admission_number'},
                    {'name': 'Student Name', 'id': 'student_name'},
                    {'name': 'Class', 'id': 'class'},
                    {'name': 'Section', 'id': 'section'}
                ],
                style_table={'margin-top': '20px'}
            )
        ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [115]:
# Define app with URL routing
app = dash.Dash(__name__, suppress_callback_exceptions=True)

# Create navigation bar
def navigation_bar():
    return html.Div([
        html.A("Student Details", href='/student-details', style=NAV_LINK_STYLE),
        html.A("Student Class Details", href='/student-class-details', style=NAV_LINK_STYLE),
        html.A("Transport Details", href='/transport-details', style=NAV_LINK_STYLE),
        html.A("Fee Details", href='/fee-details', style=NAV_LINK_STYLE),
        html.A("Fee Payment Details", href='/fee-payment-details', style=NAV_LINK_STYLE),
        html.A("View Records", href='/view-records', style=NAV_LINK_STYLE),
    ], style=NAV_BAR_STYLE)

# Set app layout with navigation bar and page content
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),  # Tracks current URL
    navigation_bar(),
    html.Div(id='page-content')
])

# Update page content based on URL
@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)

def display_page(pathname):
    if pathname == '/student-details':
        return student_details_layout()
    elif pathname == '/student-class-details':
        return student_class_details_layout()
    elif pathname == '/transport-details':
        return transport_details_layout()
    elif pathname == '/fee-details':
        return fee_details_layout()
    elif pathname == '/view-records':
        return view_records_layout()
    elif pathname == '/fee-payment-details':
        return fee_type_selection_layout()
    elif pathname == '/':
        return html.Div([
            html.H1("Welcome to the Student Management System", style=HEADER_STYLE),
            html.P("Use the navigation bar above to explore different pages.", style={'text-align': 'center'})
        ], style=APP_STYLE)
    else:
        return html.Div([
            html.H1('404: Page not found', style=HEADER_STYLE),
            html.P('The page you are looking for does not exist.', style={'text-align': 'center'})
        ], style=APP_STYLE)

In [116]:
# Toggle transport inputs based on transport usage selection
@app.callback(
    [
        Output('transport_fee', 'disabled'),
        Output('transport_fee_concession', 'disabled'),
    ],
    Input('transport_used', 'value')
)
def toggle_transport_inputs(transport_used):
    if transport_used == 'No':
        return True, True  # Disable fields
    return False, False  # Enable fields

In [117]:
@app.callback(
    Output('student-details-submit-button', 'children'),
    inputs=[
        Input('student-details-submit-button', 'n_clicks'),
        State('admission_number', 'value'),
        State('StudentName', 'value'),
        State('FatherName', 'value'),
        State('gender', 'value'),
        State('aadhar_number', 'value'),
        State('dob', 'date'),
        State('doj', 'date'),
        State('contact_number', 'value'),
        State('village', 'value'),
    ]
)
def upload_student_data(n_clicks, admission_number, student_name, guardian_name, gender, aadhar_number, date_of_birth, date_of_joining, contact_number, village):
    if n_clicks > 0:
        try:
            # Validate required fields
            if not all([admission_number, student_name, guardian_name, gender, date_of_birth, date_of_joining]):
                return "Please fill all required fields"

            # Connect to the database
            conn = get_db_connection()
            cursor = conn.cursor()

            # Check if admission number already exists
            check_query = "SELECT COUNT(*) FROM Student WHERE admission_number = %s or aadhar_number = %s"
            cursor.execute(check_query, (admission_number, aadhar_number))
            count = cursor.fetchone()[0]

            if count > 0:
                return "Admission number or Aadhar number already exists. Please use unique values."

            # SQL query to insert data
            insert_query = """
                INSERT INTO Student (
                    admission_number, student_name, guardian_name, gender, aadhar_number,
                    date_of_birth, date_of_joining, contact_number, village
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (admission_number) DO NOTHING
            """

            # Execute query with the submitted data
            cursor.execute(insert_query, (
                admission_number, student_name, guardian_name, gender,
                aadhar_number, date_of_birth, date_of_joining, contact_number, village
            ))

            # Commit changes
            conn.commit()

            # Close connection
            cursor.close()
            conn.close()

            return "Data submitted successfully!"

        except Exception as e:
            return f"Error: {str(e)}"

    return "Submit"

In [118]:
@app.callback(
    Output('student-class-submit-button', 'children'),
    inputs=[
        Input('student-class-submit-button', 'n_clicks'),
        State('admission_number', 'value'),
        State('roll_number', 'value'),
        State('photo_id', 'value'),
        State('class_no', 'value'),
        State('section', 'value'),
        State('current_year', 'value'),
        State('enrolled', 'value'),
        State('language', 'value'),
        State('vocational', 'value'),
    ]
)
def upload_student_data(n_clicks, admission_number, roll_number, photo_id, class_no, section, current_year, enrolled,language,vocational):
    if n_clicks > 0:
        try:
            # Validate required fields
            if not all([admission_number, roll_number, photo_id, class_no, section, current_year, enrolled]):
                return "Please fill all required fields"
            
            # Convert enrolled to boolean
            currently_enrolled = True if enrolled == 'Yes' else False
            
            # Connect to the database
            conn = get_db_connection()
            cursor = conn.cursor()

            # Check if admission number already exists
            check_query = "SELECT COUNT(*) FROM ClassDetails WHERE admission_number = %s and year = %s"
            cursor.execute(check_query, (admission_number, current_year))
            count = cursor.fetchone()[0]

            if count > 0:
                return "Admission number or Aadhar number already exists. Please use unique values."

            # SQL query to insert data
            insert_query = """
                INSERT INTO ClassDetails (admission_number, year, class, section, roll_number, photo_id, currently_enrolled,language,vocational)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (admission_number, year) DO UPDATE
                SET roll_number = EXCLUDED.roll_number,
                    photo_id = EXCLUDED.photo_id,
                    currently_enrolled = EXCLUDED.currently_enrolled,
                    language = EXCLUDED.language,
                    vocational = EXCLUDED.vocational
            """

            # Execute query with the submitted data
            cursor.execute(insert_query, (
                admission_number, current_year, class_no, section, roll_number, photo_id, currently_enrolled,language,vocational
            ))

            # Commit changes
            conn.commit()

            # Close connection
            cursor.close()
            conn.close()

            return "Data submitted successfully!"

        except Exception as e:
            return f"Error: {str(e)}"

    return "Submit"

In [119]:
# Define the callback to handle Fee Details form submission
@app.callback(
    Output('fee-details-submit-button', 'children'),
    inputs=[
        Input('fee-details-submit-button', 'n_clicks')
    ],
    state=[
        State('admission_number', 'value'),
        State('year', 'value'),
        State('school_fee', 'value'),
        State('school_fee_concession_reason', 'value'),
        State('transport_used', 'value'),
        # State('pickup_point', 'value'),
        # State('transport_route', 'value'),
        State('transport_fee', 'value'),
        State('transport_fee_concession', 'value'),
        State('application_fee', 'value')
    ]
)
def submit_fee_details(n_clicks, admission_number, year, school_fee, school_fee_concession_reason, transport_used, transport_fee, transport_fee_concession, application_fee):
    if n_clicks > 0:
        try:
            # Validate inputs
            if not all([admission_number, year, school_fee, application_fee]):
                return "Error: Required fields are missing."

            # Convert transport_used to boolean
            transport_used_boolean = True if transport_used == 'Yes' else False

            # Establish a database connection
            conn = get_db_connection()
            cursor = conn.cursor()

            # Insert data into the Fee table
            insert_fee_query = """
                INSERT INTO Fee (
                    admission_number, year, school_fee, concession_reason, application_fee, transport_fee, transport_fee_concession
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (admission_number, year) DO UPDATE
                SET school_fee = EXCLUDED.school_fee,
                    concession_reason = EXCLUDED.concession_reason,
                    application_fee = EXCLUDED.application_fee,
                    transport_fee = EXCLUDED.transport_fee,
                    transport_fee_concession = EXCLUDED.transport_fee_concession;
            """

            cursor.execute(insert_fee_query, (
                admission_number, year, school_fee, school_fee_concession_reason, application_fee, transport_fee, transport_fee_concession
            ))

            # Insert data into the Transport table if transport is used
            # if transport_used_boolean:
            #     insert_transport_query = """
            #         INSERT INTO Transport (transport_used, pick_up_point, route_number)
            #         VALUES (%s, %s, %s)
            #     """
            #     cursor.execute(insert_transport_query, (transport_used_boolean, pickup_point, transport_route))

            # Commit the transaction and close the connection
            conn.commit()
            cursor.close()
            conn.close()

            return "Fee details successfully uploaded!"

        except Exception as e:
            return f"Error: {str(e)}"

    return "Submit"


@app.callback(
    Output('student-details-submit-button', 'children'),
    inputs=[
        Input('student-details-submit-button', 'n_clicks'),
        State('admission_number', 'value'),
        State('StudentName', 'value'),
        State('FatherName', 'value'),
        State('gender', 'value'),
        State('aadhar_number', 'value'),
        State('dob', 'date'),
        State('doj', 'date'),
        State('contact_number', 'value'),
        State('village', 'value'),
    ]
)
def upload_student_data(n_clicks, admission_number, student_name, guardian_name, gender, aadhar_number, date_of_birth, date_of_joining, contact_number, village):
    if n_clicks > 0:
        try:
            # Validate required fields
            if not all([admission_number, student_name, guardian_name, gender, date_of_birth, date_of_joining]):
                return "Please fill all required fields"

            # Connect to the database
            conn = get_db_connection()
            cursor = conn.cursor()

            # Check if admission number already exists
            check_query = "SELECT COUNT(*) FROM Student WHERE admission_number = %s or aadhar_number = %s"
            cursor.execute(check_query, (admission_number, aadhar_number))
            count = cursor.fetchone()[0]

            if count > 0:
                return "Admission number or Aadhar number already exists. Please use unique values."

            # SQL query to insert data
            insert_query = """
                INSERT INTO Student (
                    admission_number, student_name, guardian_name, gender, aadhar_number,
                    date_of_birth, date_of_joining, contact_number, village
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (admission_number) DO NOTHING
            """

            # Execute query with the submitted data
            cursor.execute(insert_query, (
                admission_number, student_name, guardian_name, gender,
                aadhar_number, date_of_birth, date_of_joining, contact_number, village
            ))

            # Commit changes
            conn.commit()

            # Close connection
            cursor.close()
            conn.close()

            return "Data submitted successfully!"

        except Exception as e:
            return f"Error: {str(e)}"

    return "Submit"

In [120]:
if __name__ == '__main__':
    app.run_server(debug=True)